# Qcodes loading

In [1]:
# -*- coding: utf-8 -*-
"""
Created on January 27 2022

@author: FB
"""

import matplotlib.pyplot as plt
import numpy as np
import os
from time import sleep, monotonic

import qcodes as qc
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_id
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.utils.dataset.doNd import do0d,do1d, do2d

qc.config.dataset.dond_plot = True

database_name = 'FF1B_2022_OPX' #database name file. Look at cell below for its location

In [2]:
dir_database = 'D:\database'
database_format = '.db'
database_path = os.path.join(dir_database, database_name + database_format)

qc.config.user.mainfolder = os.path.join(dir_database, database_name) #saves plots in png and pdf in the database folder
qc.logger.start_all_logging()
station = qc.Station()

qc.initialise_or_create_database_at(database_path) 
exp = load_or_create_experiment(
    experiment_name= 'T7_3',
    sample_name="FF1B"
)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\T7_3\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\T7_3\.qcodes\logs\220827-17276-qcodes.log


In [3]:
STATION = qc.Station(config_file='config_20220506.yaml',use_monitor='True')

In [4]:
#qdac=STATION.load_instrument('qdac_B')
qdac_2=STATION.load_instrument('qdac_B_2')

2022-08-27 12:08:07,068 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ c:\Users\T7_3\miniconda3\envs\livestream\lib\site-packages\qcodes\station.py:568: QCoDeSDeprecationWarning: The use of a comma separated string for the limits keyword is deprecated. Use "an array like "[lower_lim, upper_lim]"" as an alternative.
  issue_deprecation_warning(



In [5]:
step_att=STATION.load_instrument('step_att_2')
print(step_att.attenuation())

Connected to: Mini-Circuits MN=RCDAT-6000-60 (serial:SN=11909040004, firmware:E5-2) in 0.06s
36.0


# Setting up buffered acquisition

In [6]:
from config_OPX_20220610_buffered import config
from qm.qua import wait_for_trigger,reset_phase,program,update_frequency,for_,stream_processing,declare,declare_stream,wait,measure,play,save,fixed,demod,ramp,amp,if_,elif_,else_,align, ramp_to_zero
from qm.QuantumMachinesManager import QuantumMachinesManager

qmm=QuantumMachinesManager(host='192.168.15.128',port=80)
        #qmm.close_all_quantum_machines()
qm=qmm.open_qm(config, close_other_machines=True)

2022-08-27 12:08:10,001 - qm - INFO - Performing health check
2022-08-27 12:08:10,011 - qm - INFO - Health check passed


In [7]:
def setup_buffered(steps=100):
    with program() as triggered_meas:
        I =declare(fixed)
        Q = declare(fixed)  
        I_stream = declare_stream()
        Q_stream = declare_stream()
        
        n = declare(int)
        wait_for_trigger('bottom_right_DQD_readout')
        with for_(n,0,n<steps,n+1):
            measure('readout_pulse_3ms','bottom_right_DQD_readout',None,demod.full('cos', I), demod.full('sin', Q))
            save(I,I_stream)
            save(Q,Q_stream)

        with stream_processing():
            I_stream.buffer(steps).save('I')
            Q_stream.buffer(steps).save('Q')

    triggered_id = qm.compile(triggered_meas)
    return triggered_id

In [8]:
def start_triggered(triggered_id):
    p_job=qm.queue.add_compiled(triggered_id)
    return p_job.wait_for_execution()

def get_amp(job):
    I_handle = job.result_handles.get('I')
    Q_handle = job.result_handles.get('Q')
    Q_handle.wait_for_all_values()
    I_handle.wait_for_all_values()
    results_I = I_handle.fetch_all()
    results_Q = Q_handle.fetch_all()
    return np.sqrt(results_Q**2+results_I**2)


def get_phase(job):
    I_handle = job.result_handles.get('I')
    Q_handle = job.result_handles.get('Q')
    Q_handle.wait_for_all_values()
    I_handle.wait_for_all_values()
    results_I = I_handle.fetch_all()
    results_Q = Q_handle.fetch_all()
    return np.arctan2(results_Q,results_I)

from qcodes import Measurement
from time import sleep
import time
import re
# do2d_buffered takes as channel not (e.g.) qdac.ch01.v but qdac.ch01, as I need later (in ramp_voltages_2d_buffered) the channel number (obtained by int(re.findall("\d+", <string name>)[0]))
def do1d_buffered(chan, vstart, vend, steps ,step_length, job_id):    
    
    #dmm_lockin.sample.pretrigger_count(0)
    start = time.time()
    meas = Measurement()
    v1s = np.linspace(vstart, vend, steps)
    #v2s = np.linspace(fast_vstart, fast_vend, fast_steps)
    meas.register_parameter(chan.v)
    #meas.register_parameter(fast_chan.v)
    meas.register_custom_parameter('amp', 'amp', 'V', setpoints=(chan.v,))
    meas.register_custom_parameter('phase', 'phase', 'rad', setpoints=(chan.v,))
    ramp_time = step_length*steps
    with meas.run() as datasaver: 
        job = start_triggered(job_id)
        # sleep(2)
        
        meas_time = qdac_2.ramp_voltages( [int(re.findall("\d+", chan.short_name)[0])], [vstart], [vend], ramp_time)
                                                
        # sleep(meas_time)
        print(meas_time)
        amp_data = get_amp(job) #data = dmm.fetch()
        phase_data = get_phase(job)
        datasaver.add_result((chan.v, np.linspace(vstart,vend,steps)),
                            ('amp', amp_data.flatten()),
                            ('phase',phase_data.flatten()))    
    #dataset2D = datasaver.dataset
    end = time.time()
    # plt.pcolormesh(data)
    print(end - start)


## do1d buffered

In [9]:
# # %% 
# qdac_2.ch06.sync(1) #channel
# qdac_2.ch06.sync_delay(0)
# qdac_2.ch06.sync_duration(0.001)
# qdac_2.ch06.v.label = 'BNC 30'

# steps = 100 #number of pixel along slow axis
# time_per_pixel = 0.003 # in seconds
# triggered_id = setup_buffered(steps=steps)
# do1d_buffered(qdac_2.ch06, -0.2, -0.1 ,steps,time_per_pixel,triggered_id)
# qdac_2.BNC30(-0.150)

## 2D buffered

In [10]:
from qcodes import Measurement
from time import sleep
import time
import re
# do2d_buffered takes as channel not (e.g.) qdac.ch01.v but qdac.ch01, as I need later (in ramp_voltages_2d_buffered) the channel number (obtained by int(re.findall("\d+", <string name>)[0]))
def do2d_buffered(slow_chan, slow_vstart, slow_vend, slow_steps , fast_chan, fast_vstart, fast_vend, fast_steps ,step_length, job_id):    
    
    #dmm_lockin.sample.pretrigger_count(0)
    start = time.time()
    meas = Measurement()
    v1s = np.linspace(slow_vstart, slow_vend, slow_steps)
    v2s = np.linspace(fast_vstart, fast_vend, fast_steps)
    meas.register_parameter(slow_chan.v)
    meas.register_parameter(fast_chan.v)
    meas.register_custom_parameter('amp', 'amp', 'V', setpoints=(slow_chan.v,fast_chan.v,))
    meas.register_custom_parameter('phase', 'phase', 'rad', setpoints=(slow_chan.v,fast_chan.v,))
    
    with meas.run() as datasaver: 
        job = start_triggered(job_id)
        # sleep(2)
        meas_time = qdac_2.ramp_voltages_2d( slow_chans=[int(re.findall("\d+", slow_chan.short_name)[0])], slow_vstart=[slow_vstart], slow_vend=[slow_vend],
                                    fast_chans=[int(re.findall("\d+", fast_chan.short_name)[0])], fast_vstart=[fast_vstart], fast_vend=[fast_vend],
                                    step_length=step_length,slow_steps = slow_steps, fast_steps = fast_steps)
                                                
        # sleep(meas_time)
        print(meas_time)

        amp_data =  get_amp(job)  #data = dmm.fetch()
        phase_data = get_phase(job)
        datasaver.add_result((slow_chan.v, np.repeat(v1s,fast_steps)),
                            (fast_chan.v, np.tile(v2s,slow_steps)),
                            ('amp', amp_data.flatten()),
                            ('phase',phase_data.flatten()))    
    # #dataset2D = datasaver.dataset
    end = time.time()
    # # plt.pcolormesh(data)
    print(end - start)


In [11]:
qdac_2.ch12.sync(1) #slow channel
qdac_2.ch12.sync_delay(0)
qdac_2.ch12.sync_duration(0.001)
qdac_2.ch12.v.label = 'BNC 36'
qdac_2.ch05.v.label = 'BNC 29'

In [11]:
qdac_2.BNC34()

-0.365005

In [12]:
qdac_2.BNC29()

-0.126691

In [13]:
qdac_2.BNC36()

-0.172801

In [14]:
qdac_2.BNC32()

-0.519996

In [15]:
qdac_2.BNC37()

-0.699991

In [16]:
qdac_2.BNC30()

-0.132105

In [17]:
#qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-07-28 09:48:52,273 - qm - INFO - Flags: 
2022-07-28 09:48:52,274 - qm - INFO - Compiling program
Starting experimental run with id: 809. 
2022-07-28 09:49:21,969 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.13787007331848


In [18]:
qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-07-28 09:50:21,218 - qm - INFO - Flags: 
2022-07-28 09:50:21,219 - qm - INFO - Compiling program
Starting experimental run with id: 810. 
2022-07-28 09:50:44,002 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.26127815246582


In [19]:
ch2_DC_bias = -0.1265 #BNC29 in V
ch1_DC_bias = -0.1725 #BNC36 in V

ch2_DC_initialize = -0.1238 #BNC29 in V
ch1_DC_initialize = -0.1829 #BNC36 in V

In [20]:
qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

# 29 Jul 2022

In [11]:
#qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-07-29 08:41:57,390 - qm - INFO - Flags: 
2022-07-29 08:41:57,391 - qm - INFO - Compiling program
Starting experimental run with id: 811. 
2022-07-29 08:42:30,666 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.164165496826172


In [12]:
qdac_2.BNC30()

-0.131991

In [15]:
qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 814. 
2022-07-29 08:47:44,755 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.06813621520996


In [18]:
ch2_DC_bias = -0.1271 #BNC29 in V
ch1_DC_bias = -0.1731 #BNC36 in V

ch2_DC_initialize = -0.123 #BNC29 in V
ch1_DC_initialize = -0.1851 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

In [13]:
#qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-07-29 15:49:58,255 - qm - INFO - Flags: 
2022-07-29 15:49:58,257 - qm - INFO - Compiling program
Starting experimental run with id: 815. 
2022-07-29 15:50:27,248 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.192787170410156


In [16]:
ch2_DC_bias = -0.1274 #BNC29 in V
ch1_DC_bias = -0.1728 #BNC36 in V

ch2_DC_initialize = -0.1235 #BNC29 in V
ch1_DC_initialize = -0.183 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

# 30 Jul 2022

In [11]:
#qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-07-30 13:54:20,266 - qm - INFO - Flags: 
2022-07-30 13:54:20,267 - qm - INFO - Compiling program
Starting experimental run with id: 826. 
2022-07-30 13:54:53,622 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.12001061439514


In [13]:
qdac_2.BNC30()

-0.132105

In [14]:
qdac_2.BNC30(-0.1319)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 827. 
2022-07-30 13:55:59,907 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.247979402542114


In [15]:
ch2_DC_bias = -0.1265 #BNC29 in V
ch1_DC_bias = -0.1726 #BNC36 in V

ch2_DC_initialize = -0.1235 #BNC29 in V
ch1_DC_initialize = -0.183 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

# 1 Aug 2022

In [10]:
qdac_2.BNC29()


-0.1265

In [11]:
qdac_2.BNC36()

-0.172592

In [12]:
qdac_2.BNC30()

-0.131895

In [13]:
#qdac_2.BNC30(-0.1319)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-01 19:21:31,867 - qm - INFO - Flags: 
2022-08-01 19:21:31,868 - qm - INFO - Compiling program
Starting experimental run with id: 888. 
2022-08-01 19:22:01,323 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.121402502059937


In [14]:
qdac_2.BNC29(-0.1266)
qdac_2.BNC36(-0.1725)

In [15]:
qdac_2.BNC29(-0.127)
qdac_2.BNC36(-0.1733)

In [17]:
step_att.attenuation(34)

# 3 aug 2022

In [12]:
#qdac_2.BNC30(-0.1319)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-03 08:00:48,526 - qm - INFO - Flags: 
2022-08-03 08:00:48,526 - qm - INFO - Compiling program
Starting experimental run with id: 904. 
2022-08-03 08:01:18,285 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.170734882354736


In [13]:
qdac_2.BNC30()

-0.131895

In [14]:
qdac_2.BNC30(-0.1316)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 905. 
2022-08-03 08:02:29,343 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.557135581970215


In [12]:
qdac_2.BNC30(-0.1317)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-03 13:42:50,468 - qm - INFO - Flags: 
2022-08-03 13:42:50,470 - qm - INFO - Compiling program
Starting experimental run with id: 922. 
2022-08-03 13:43:18,745 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.206955671310425


In [16]:
qdac_2.BNC29(-0.1265)
qdac_2.BNC36(-0.1723)

In [14]:
qdac_2.BNC30(-0.1325)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 924. 
2022-08-03 13:45:27,178 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.09781765937805


In [15]:
qdac_2.BNC30(-0.1323)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 925. 
2022-08-03 13:46:15,590 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.230884790420532


In [17]:
qdac_2.BNC29(-0.1272)
qdac_2.BNC36(-0.1738)

# 4 Aug 2022

In [12]:
qdac_2.BNC34()

-0.365005

In [13]:
#qdac_2.BNC30(-0.1323)
#qdac_2.BNC32(-0.520)
qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-04 10:39:46,158 - qm - INFO - Flags: 
2022-08-04 10:39:46,159 - qm - INFO - Compiling program
Starting experimental run with id: 948. 
2022-08-04 10:40:18,235 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.317617654800415


In [17]:
qdac_2.BNC30(-0.1322)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.20,-0.130, slow_steps, qdac_2.ch05, -0.17, -0.11, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 952. 
2022-08-04 10:49:08,102 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.376784801483154


In [18]:
ch2_DC_bias = -0.1294 #BNC29 in V
ch1_DC_bias = -0.1775 #BNC36 in V

ch2_DC_initialize = -0.1141 #BNC29 in V
ch1_DC_initialize = -0.1965 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

In [12]:
qdac_2.BNC30(-0.1323)
#qdac_2.BNC32(-0.520)
qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-04 11:01:47,781 - qm - INFO - Flags: 
2022-08-04 11:01:47,781 - qm - INFO - Compiling program
Starting experimental run with id: 954. 
2022-08-04 11:02:08,795 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.14810013771057


In [13]:
ch2_DC_bias = -0.1272 #BNC29 in V
ch1_DC_bias = -0.1738 #BNC36 in V

ch2_DC_initialize = -0.123 #BNC29 in V
ch1_DC_initialize = -0.1856 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

In [11]:
qdac_2.BNC34()


-0.365005

In [12]:
#qdac_2.BNC30(-0.1323)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-04 11:26:57,423 - qm - INFO - Flags: 
2022-08-04 11:26:57,424 - qm - INFO - Compiling program
Starting experimental run with id: 955. 
2022-08-04 11:27:18,704 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.191243171691895


In [13]:
step_att.attenuation(36)

In [14]:
qdac_2.BNC30()

-0.132296

In [14]:
qdac_2.BNC30(-0.1323)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-04 12:03:07,040 - qm - INFO - Flags: 
2022-08-04 12:03:07,041 - qm - INFO - Compiling program
Starting experimental run with id: 960. 
2022-08-04 12:03:40,568 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.92226791381836


In [19]:
ch2_DC_bias = -0.1275 #BNC29 in V
ch1_DC_bias = -0.1735 #BNC36 in V

ch2_DC_initialize = -0.1228 #BNC29 in V
ch1_DC_initialize = -0.1858 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

In [17]:
step_att.attenuation(36)

In [15]:
qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 965. 
2022-08-04 14:48:06,445 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.110247135162354


In [16]:
ch2_DC_bias = -0.1275 #BNC29 in V
ch1_DC_bias = -0.1735 #BNC36 in V

ch2_DC_initialize = -0.1228 #BNC29 in V
ch1_DC_initialize = -0.1858 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

In [19]:
step_att.attenuation(36)

In [12]:
qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-04 18:24:00,157 - qm - INFO - Flags: 
2022-08-04 18:24:00,158 - qm - INFO - Compiling program
Starting experimental run with id: 968. 
2022-08-04 18:24:24,497 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.151594638824463


In [16]:
ch2_DC_bias = -0.1279 #BNC29 in V
ch1_DC_bias = -0.174 #BNC36 in V

ch2_DC_initialize = -0.1228 #BNC29 in V
ch1_DC_initialize = -0.1858 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

In [12]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-04 22:29:31,764 - qm - INFO - Flags: 
2022-08-04 22:29:31,765 - qm - INFO - Compiling program
Starting experimental run with id: 969. 
2022-08-04 22:29:56,861 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.17161536216736


In [16]:
ch2_DC_bias = -0.1264 #BNC29 in V
ch1_DC_bias = -0.1742 #BNC36 in V

ch2_DC_initialize =-0.1269 #BNC29 in V
ch1_DC_initialize = -0.1771 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

# 5 Aug 2022

In [16]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-05 09:18:55,759 - qm - INFO - Flags: 
2022-08-05 09:18:55,760 - qm - INFO - Compiling program
Starting experimental run with id: 971. 
2022-08-05 09:19:29,069 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.307960510253906


In [18]:
ch2_DC_bias = -0.1269 #BNC29 in V
ch1_DC_bias = -0.1739 #BNC36 in V

ch2_DC_initialize =-0.1261 #BNC29 in V
ch1_DC_initialize = -0.1786 #BNC36 in V

qdac_2.BNC29(ch2_DC_bias)
qdac_2.BNC36(ch1_DC_bias)

In [12]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-05 17:51:49,264 - qm - INFO - Flags: 
2022-08-05 17:51:49,265 - qm - INFO - Compiling program
Starting experimental run with id: 975. 
2022-08-05 17:52:18,766 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.267558574676514


In [16]:
gate_29_DC_bias = -0.1281 #BNC29 in V
gate_36_DC_bias = -0.1735 #BNC36 in V

gate_29_DC_initialize =-0.126 #BNC29 in V
gate_36_DC_initialize = -0.1778 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 6 Aug 2022

In [12]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-06 11:32:58,199 - qm - INFO - Flags: 
2022-08-06 11:32:58,200 - qm - INFO - Compiling program
Starting experimental run with id: 978. 
2022-08-06 11:33:27,913 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.145816564559937


In [13]:
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.174 #BNC36 in V

gate_29_DC_initialize =-0.1257 #BNC29 in V
gate_36_DC_initialize = -0.1782 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [11]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-06 23:31:17,841 - qm - INFO - Flags: 
2022-08-06 23:31:17,842 - qm - INFO - Compiling program
Starting experimental run with id: 998. 
2022-08-06 23:31:46,989 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.167484998703003


In [12]:
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.174 #BNC36 in V

gate_29_DC_initialize =-0.1257 #BNC29 in V
gate_36_DC_initialize = -0.1782 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 8 Aug 2022

In [12]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-08 08:06:18,079 - qm - INFO - Flags: 
2022-08-08 08:06:18,079 - qm - INFO - Compiling program
Starting experimental run with id: 1024. 
2022-08-08 08:06:46,213 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.103577852249146


In [13]:
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.174 #BNC36 in V

gate_29_DC_initialize =-0.1267 #BNC29 in V
gate_36_DC_initialize = -0.1759 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [12]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-08 13:19:43,665 - qm - INFO - Flags: 
2022-08-08 13:19:43,665 - qm - INFO - Compiling program
Starting experimental run with id: 1026. 
2022-08-08 13:20:16,814 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.211599588394165


In [15]:
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.174 #BNC36 in V

gate_29_DC_initialize =-0.1267 #BNC29 in V
gate_36_DC_initialize = -0.1759 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [14]:
qdac_2.BNC29(-0.150)
qdac_2.BNC36(-0.154)

In [ ]:
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.174 #BNC36 in V

gate_29_DC_initialize =-0.1267 #BNC29 in V
gate_36_DC_initialize = -0.1759 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [11]:
#qdac_2.BNC30(-0.13225)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-08 15:08:07,945 - qm - INFO - Flags: 
2022-08-08 15:08:07,946 - qm - INFO - Compiling program
Starting experimental run with id: 1027. 
2022-08-08 15:08:35,898 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.07412338256836


In [12]:
qdac_2.BNC30()

-0.132258

In [13]:
#qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-09 14:32:53,156 - qm - INFO - Flags: 
2022-08-09 14:32:53,157 - qm - INFO - Compiling program
Starting experimental run with id: 1029. 
2022-08-09 14:33:22,070 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.298847913742065


In [14]:
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.174 #BNC36 in V

gate_29_DC_initialize =-0.1267 #BNC29 in V
gate_36_DC_initialize = -0.1759 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 9 Aug 2022

In [14]:
qdac_2.BNC30()

-0.132105

In [15]:
qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-09 14:40:05,485 - qm - INFO - Flags: 
2022-08-09 14:40:05,487 - qm - INFO - Compiling program
Starting experimental run with id: 1030. 
2022-08-09 14:40:26,776 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.021841287612915


In [16]:
qdac_2.BNC30(-0.13205)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-09 14:41:28,010 - qm - INFO - Flags: 
2022-08-09 14:41:28,011 - qm - INFO - Compiling program
Starting experimental run with id: 1031. 
2022-08-09 14:41:49,065 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.119899034500122


In [19]:
gate_29_DC_bias = -0.1277 #BNC29 in V
gate_36_DC_bias = -0.1738 #BNC36 in V

gate_29_DC_initialize =-0.1257 #BNC29 in V
gate_36_DC_initialize = -0.1773 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 10 Aug 2022

In [12]:
#qdac_2.BNC30(-0.13205)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-10 08:43:49,216 - qm - INFO - Flags: 
2022-08-10 08:43:49,218 - qm - INFO - Compiling program
Starting experimental run with id: 1033. 
2022-08-10 08:44:18,426 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.172198057174683


In [14]:
gate_29_DC_bias = -0.1264 #BNC29 in V
gate_36_DC_bias = -0.1738 #BNC36 in V

gate_29_DC_initialize =-0.1257 #BNC29 in V
gate_36_DC_initialize = -0.1778 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [12]:
#qdac_2.BNC30(-0.13205)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-10 16:01:49,293 - qm - INFO - Flags: 
2022-08-10 16:01:49,294 - qm - INFO - Compiling program
Starting experimental run with id: 1034. 
2022-08-10 16:02:20,062 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.10712456703186


In [13]:
qdac_2.BNC30()


-0.132048

In [14]:
qdac_2.BNC30(-0.1319)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 1035. 
2022-08-10 16:03:41,555 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.11363911628723


In [15]:
gate_29_DC_bias = -0.1264 #BNC29 in V
gate_36_DC_bias = -0.1738 #BNC36 in V

gate_29_DC_initialize =-0.1257 #BNC29 in V
gate_36_DC_initialize = -0.1778 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 13 Aug 2022

In [18]:
#qdac_2.BNC30(-0.1319)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-13 12:34:15,502 - qm - INFO - Flags: 
2022-08-13 12:34:15,503 - qm - INFO - Compiling program
Starting experimental run with id: 1060. 
2022-08-13 12:34:49,821 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.263610363006592


In [ ]:
gate_29_DC_bias = -0.1264 #BNC29 in V
gate_36_DC_bias = -0.1738 #BNC36 in V

gate_29_DC_initialize =-0.1257 #BNC29 in V
gate_36_DC_initialize = -0.1778 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [19]:
qdac_2.BNC30()

-0.131895

In [20]:
qdac_2.BNC30(-0.1318)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 1061. 
2022-08-13 12:36:04,256 - qm - INFO - Sending pre-compiled program to QOP
30.0
30.983752250671387


In [21]:
gate_29_DC_bias = -0.1261 #BNC29 in V
gate_36_DC_bias = -0.1728 #BNC36 in V

gate_29_DC_initialize =-0.1255 #BNC29 in V
gate_36_DC_initialize = -0.1752 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [12]:
qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-13 18:07:06,767 - qm - INFO - Flags: 
2022-08-13 18:07:06,768 - qm - INFO - Compiling program
Starting experimental run with id: 1067. 
2022-08-13 18:07:38,991 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.17874050140381


In [13]:
gate_29_DC_bias = -0.1261 #BNC29 in V
gate_36_DC_bias = -0.1728 #BNC36 in V

gate_29_DC_initialize =-0.1255 #BNC29 in V
gate_36_DC_initialize = -0.1752 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [14]:
qdac_2.BNC30(-0.1318)


In [15]:
gate_29_DC_bias = -0.1261 #BNC29 in V
gate_36_DC_bias = -0.1728 #BNC36 in V

gate_29_DC_initialize =-0.1255 #BNC29 in V
gate_36_DC_initialize = -0.1752 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [12]:
#qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-14 10:54:03,647 - qm - INFO - Flags: 
2022-08-14 10:54:03,648 - qm - INFO - Compiling program
Starting experimental run with id: 1069. 
2022-08-14 10:54:32,872 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.219451189041138


In [13]:
qdac_2.BNC30()

-0.1318

In [14]:
qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

Starting experimental run with id: 1070. 
2022-08-14 10:55:33,847 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.104364156723022


In [15]:
gate_29_DC_bias = -0.12785 #BNC29 in V
gate_36_DC_bias = -0.1739 #BNC36 in V

# gate_29_DC_initialize =-0.1255 #BNC29 in V
# gate_36_DC_initialize = -0.1752 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

after running metastable 2D map 

In [18]:
gate_29_DC_bias = -0.12785 #BNC29 in V
gate_36_DC_bias = -0.1739  #BNC36 in V

gate_29_DC_initialize = gate_29_DC_bias #BNC29 in V
gate_36_DC_initialize = gate_29_DC_bias #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [12]:
#qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
qdac_2.BNC29(-0.1267)
qdac_2.BNC36(-0.1728)

2022-08-14 12:12:31,921 - qm - INFO - Flags: 
2022-08-14 12:12:31,922 - qm - INFO - Compiling program
Starting experimental run with id: 1073. 
2022-08-14 12:13:00,716 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.069989919662476


In [16]:
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

gate_29_DC_initialize = -0.1263 #BNC29 in V
gate_36_DC_initialize = -0.1759 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 15 Aug 2022

In [12]:
#qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

2022-08-15 17:19:19,863 - qm - INFO - Flags: 
2022-08-15 17:19:19,864 - qm - INFO - Compiling program
Starting experimental run with id: 1074. 
2022-08-15 17:19:50,073 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.190763235092163


In [13]:
qdac_2.BNC30()


-0.131991

In [14]:
qdac_2.BNC30(-0.1319)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.365)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.190,-0.120, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

Starting experimental run with id: 1075. 
2022-08-15 17:39:19,229 - qm - INFO - Sending pre-compiled program to QOP
30.0
30.977296113967896


In [15]:
gate_29_DC_bias = -0.12671 #BNC29 in V
gate_36_DC_bias = -0.1731  #BNC36 in V

gate_29_DC_initialize = -0.1263 #BNC29 in V
gate_36_DC_initialize = -0.1747 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [16]:
gate_29_DC_bias = -0.1271 #BNC29 in V
gate_36_DC_bias = -0.1735  #BNC36 in V

gate_29_DC_initialize = -0.1263 #BNC29 in V
gate_36_DC_initialize = -0.1747 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 17 Aug 2022

In [11]:
qdac_2.BNC34()

-0.365005

In [12]:
#qdac_2.BNC30(-0.1319)
#qdac_2.BNC32(-0.520)
qdac_2.BNC34(-0.355)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.19, -0.13, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

2022-08-17 16:56:39,613 - qm - INFO - Flags: 
2022-08-17 16:56:39,614 - qm - INFO - Compiling program
Starting experimental run with id: 1091. 
2022-08-17 16:57:08,878 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.154223203659058


In [13]:
qdac_2.BNC30()

-0.131895

In [14]:
qdac_2.BNC30(-0.1318)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.355)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

Starting experimental run with id: 1092. 
2022-08-17 16:58:49,405 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.141907215118408


In [15]:
gate_29_DC_bias = -0.1314 #BNC29 in V
gate_36_DC_bias = -0.1809  #BNC36 in V

gate_29_DC_initialize = -0.1306 #BNC29 in V
gate_36_DC_initialize = -0.1831 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 18 Aug 2022

In [12]:
#qdac_2.BNC30(-0.1318)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.355)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

2022-08-18 08:58:25,540 - qm - INFO - Flags: 
2022-08-18 08:58:25,541 - qm - INFO - Compiling program
Starting experimental run with id: 1093. 
2022-08-18 08:58:54,039 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.211315631866455


In [14]:
gate_29_DC_bias = -0.1314 #BNC29 in V
gate_36_DC_bias = -0.1809  #BNC36 in V

gate_29_DC_initialize = -0.1306 #BNC29 in V
gate_36_DC_initialize = -0.1831 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 20 Aug 2022

In [12]:
#qdac_2.BNC30(-0.1318)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.355)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

2022-08-20 09:52:30,414 - qm - INFO - Flags: 
2022-08-20 09:52:30,415 - qm - INFO - Compiling program
Starting experimental run with id: 1101. 
2022-08-20 09:52:59,164 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.791581630706787


In [14]:
gate_29_DC_bias = -0.1303 #BNC29 in V
gate_36_DC_bias = -0.1812  #BNC36 in V

# gate_29_DC_initialize = -0.1306 #BNC29 in V
# gate_36_DC_initialize = -0.1831 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [12]:
qdac_2.BNC34()

-0.354994

In [13]:
#qdac_2.BNC30(-0.1318)
#qdac_2.BNC32(-0.520)
qdac_2.BNC34(-0.345)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

2022-08-20 11:24:37,702 - qm - INFO - Flags: 
2022-08-20 11:24:37,703 - qm - INFO - Compiling program
Starting experimental run with id: 1102. 
2022-08-20 11:25:10,805 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.125422716140747


In [14]:
qdac_2.BNC30(-0.1316)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.345)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

Starting experimental run with id: 1103. 
2022-08-20 11:28:31,514 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.219821214675903


In [15]:
gate_29_DC_bias = -0.1351 #BNC29 in V
gate_36_DC_bias = -0.1888  #BNC36 in V

# gate_29_DC_initialize = -0.1306 #BNC29 in V
# gate_36_DC_initialize = -0.1831 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [11]:
#qdac_2.BNC30(-0.1316)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.345)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

2022-08-21 11:41:30,131 - qm - INFO - Flags: 
2022-08-21 11:41:30,131 - qm - INFO - Compiling program
Starting experimental run with id: 1104. 
2022-08-21 11:41:55,156 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.119699001312256


In [12]:
qdac_2.BNC30()

-0.131609

In [13]:
qdac_2.BNC30(-0.1317)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.345)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

Starting experimental run with id: 1105. 
2022-08-21 11:43:22,122 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.24916362762451


In [14]:
gate_29_DC_bias = -0.1343 #BNC29 in V
gate_36_DC_bias = -0.1885  #BNC36 in V

# gate_29_DC_initialize = -0.1306 #BNC29 in V
# gate_36_DC_initialize = -0.1831 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [ ]:
qdac_2.BNC30(-0.1317)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.345)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

In [12]:
#qdac_2.BNC30(-0.1317)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1269 #BNC29 in V
gate_36_DC_bias = -0.1733  #BNC36 in V

2022-08-21 12:07:53,182 - qm - INFO - Flags: 
2022-08-21 12:07:53,184 - qm - INFO - Compiling program
Starting experimental run with id: 1106. 
2022-08-21 12:08:24,821 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.114392042160034


In [13]:
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.177  #BNC36 in V

# gate_29_DC_initialize = -0.1306 #BNC29 in V
# gate_36_DC_initialize = -0.1831 #BNC36 in V

qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 22 Aug 2022

In [12]:
#qdac_2.BNC30(-0.1317)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.177  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

2022-08-22 17:13:49,934 - qm - INFO - Flags: 
2022-08-22 17:13:49,936 - qm - INFO - Compiling program
Starting experimental run with id: 1107. 
2022-08-22 17:14:18,816 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.231635570526123


In [13]:
qdac_2.BNC30()

-0.131705

In [16]:
qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.177  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

Starting experimental run with id: 1110. 
2022-08-22 17:20:16,244 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.144236087799072


In [17]:
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.1779  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [13]:
qdac_2.BNC30(-0.1331)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.177  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

2022-08-22 18:43:37,969 - qm - INFO - Flags: 
2022-08-22 18:43:37,970 - qm - INFO - Compiling program
Starting experimental run with id: 1115. 
2022-08-22 18:44:06,802 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.214704036712646


In [14]:
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.1779  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

In [13]:
qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.177  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

2022-08-23 18:40:28,739 - qm - INFO - Flags: 
2022-08-23 18:40:28,740 - qm - INFO - Compiling program
Starting experimental run with id: 1119. 
2022-08-23 18:40:57,815 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.267985582351685


In [14]:
gate_29_DC_bias = -0.1287 #BNC29 in V
gate_36_DC_bias = -0.1782  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

# 27 Aug 2022

In [12]:
#qdac_2.BNC30(-0.1321)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.177  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

2022-08-27 12:08:20,356 - qm - INFO - Flags: 
2022-08-27 12:08:20,357 - qm - INFO - Compiling program
Starting experimental run with id: 1120. 
2022-08-27 12:08:50,546 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.08063316345215


In [13]:
qdac_2.BNC30()

-0.132105

In [14]:
qdac_2.BNC30(-0.132)
#qdac_2.BNC32(-0.520)
#qdac_2.BNC34(-0.360)

slow_steps = 100 #number of pixel along slow axis
fast_steps = 100 #number of pixel along fast axis
time_per_pixel = 0.003 # in seconds
#triggered_id = setup_buffered(steps=fast_steps*slow_steps)

job=do2d_buffered(qdac_2.ch12,-0.200,-0.110, slow_steps, qdac_2.ch05, -0.18, -0.12, fast_steps,time_per_pixel, triggered_id)
gate_29_DC_bias = -0.1286 #BNC29 in V
gate_36_DC_bias = -0.177  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)

Starting experimental run with id: 1121. 
2022-08-27 12:10:21,180 - qm - INFO - Sending pre-compiled program to QOP
30.0
31.223459720611572


In [15]:
gate_29_DC_bias = -0.1287 #BNC29 in V
gate_36_DC_bias = -0.1791  #BNC36 in V
qdac_2.BNC29(gate_29_DC_bias)
qdac_2.BNC36(gate_36_DC_bias)